**ANGGOTA KELOMPOK**

**[1] Muhammad Reza Adi Nugraha - 1301204078**

**[2] Zahran Raihansyah Likumahua - 1301204049**

# **LIBRARY**

In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split # Untuk membagi data latih dan data uji

# **MEMBACA FILE TRAIN**

In [ ]:
excel = pd.ExcelFile('/content/traintest.xlsx')

In [ ]:
# Untuk mengubah agar range nilai setiap data sama
def normalisasi(data) :
  return (data - data.min()) / (data.max() - data.min())

In [ ]:
# Baca sheet train dan lakukan normalisasi
train_dataset = pd.read_excel(excel, sheet_name = "train")
train_dataset = normalisasi(train_dataset)
train_dataset = train_dataset.drop(columns=["id"])
train_dataset.head()

,x1,x2,x3,y
0,0.566038,0.545455,0.000000,1.0
1,0.452830,0.181818,0.211538,0.0
2,0.660377,0.363636,0.423077,0.0
3,0.075472,0.181818,0.000000,1.0
4,0.150943,1.000000,0.403846,0.0


# **TRAINING MODEL**

In [ ]:
class KNN():
  
  # Inisilisasi K yang digunakan
  def __init__(self, K):
    self.K = K

  # Inisilisasi dataset yang digunakan untuk training
  def train(self, X_train, y_train):
    self.X_train = X_train
    self.y_train = y_train

  # Melakukan prediksi data test
  def predict(self, X_test):
    # Ubah ke tipe data list
    data = X_test.values.tolist()
    hasil = []
    for i in range(len(data)):
      # Ambil data ke-i  berupa [x1, x2, x3]
      test_data = data[i]
      # Panggil fungsi predicted_label untuk mengembalikan label yang sesuai berdasrkan data ke-i
      label = self.predicted_label(test_data)
      # Label ditambahkan di list variabel hasil
      hasil.append(label)
    return hasil
  
  # Mengembalikan prediksi label
  def predicted_label(self, test_data):
    # Untuk menyimpan jarak test_data dengan semua titik di train_data dan labelnya
    data = {"jarak":[], "label":[]}

    # Ubah ke dalam tipde data list
    train_data = X_train[['x1', 'x2', 'x3']].values.tolist()
    # Painggil fungsi jarak untuk mengembalikan list jarak tes_data ke semua titik di train_data
    jarak_titik = self.jarak(train_data, test_data)

    # Mendapatkan semua label sesuai urutan data
    Z = y_train.values.tolist()
    list_label = []
    for i in range(len(Z)):
     list_label.append(Z[i][0])

    # Jadikan sebuah dataframe agar ketika diurutkan dari jarak terdekat label nya ikut menyesuaikan
    data["jarak"].append(jarak_titik)
    data["label"].append(list_label)
    data = pd.DataFrame(data)

    # Sort data berdasarkan jarak dari jarak yang terdekat
    data.sort_values(by=["jarak"])

    # Ambil list label dari data
    nearest = data["label"][0]

    # Hitung jumlah kemunculan label_1 dan label_0
    label_1 = 0
    label_0 = 0

    # Pengecekan hanya sampai ke index K-1 karena kita akan ambil K tetangga terdekat
    for i in range(self.K):
      data = nearest[i]
      if data == 1:
        label_1 +=1
      else:
        label_0 += 1

    # Jika label_1 > label_0 berarti label banyak muncul dari K tetangga terdekat adalah 1
    if label_1 > label_0:
      return 1
    # Selain itu maka label banyak muncul dari K tetangga terdekat adalah 1
    else:
      return 0

  # Mengembalikan jarak x_test dengan tiap data di x_train  
  def jarak(self, x_train, x_test):
    # Hasil jarak disimpan di list_dist
    list_dist = []
    for i in range(len(x_train)):
      # Menggunakan euclidean distance
      dist = (x_train[i][0] - x_test[0])**2 + (x_train[i][1] - x_test[1])**2 + (x_train[i][2] - x_test[2])**2
      dist = math.sqrt(dist)
      # Hasil ditambahkan ke lit_dist
      list_dist.append(dist)
    return list_dist

## **MEMBAGI TRAINING DATASET**

In [ ]:
# Ambil X dan y dari train_dataset
X = train_dataset[["x1", "x2", "x3"]]
y = train_dataset[["y"]]
# Pisahkan 70% untuk training dan 30% untuk testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=9)

## **AKURASI MODEL**

In [ ]:
# Buat model dengan K = 7
model = KNN(K=7)
# Pilih data untuk training
model.train(X_train, y_train)
# Test kan hasilnya dengan data yanng di test
hasil = model.predict(X_test)

In [ ]:
# Cek presentasi keberhasilan model memprediksi label yang tepat. Bandingkan hasil prediksi dengan label sebenarnya.
i = 0
# Hitung berapa yang benar
benar = 0
for data in y_test["y"]:
  if data == hasil[i]:
    benar+= 1
  i+= 1

# Tampilkan akurasi model yang dibuta dalam persen
print("Akurasi: {:.2%}".format(benar/len(y_test)))

Akurasi: 79.78%


# **TESTING**

In [ ]:
# Membaca data sheet test
test_dataset = pd.read_excel(excel, sheet_name = "test")
final_dataset = test_dataset
# Drop kolom y karena nilainya mau kita cari dan agar tidak merusak hasil normalisasi
test_dataset = test_dataset.drop(columns=["y"])
test_dataset = test_dataset.drop(columns=["id"])
test_dataset = normalisasi(test_dataset)
test_dataset.head()

,x1,x2,x3
0,0.04,0.090909,0.666667
1,1.00,0.727273,0.000000
2,0.64,0.181818,1.000000
3,0.28,0.454545,1.000000
4,0.12,0.181818,0.000000


In [ ]:
# Gunakan model yang sudah dilatih sebelumnya untuk melakukan prediksi untuk test_dataset
hasil = model.predict(test_dataset)

# Simpan hasilnya dijadikan dalam dataframe agar dapat disatukan dengan test_dataset
hasil = {"y":hasil}
hasil = pd.DataFrame(hasil)
final_dataset["y"] = hasil
final_dataset

,id,x1,x2,x3,y
0,297,43,59,2,1
1,298,67,66,0,1
2,299,58,60,3,1
3,300,49,63,3,1
4,301,45,60,0,1
5,302,54,58,1,1
6,303,56,66,3,1
7,304,42,69,1,1
8,305,50,59,2,1
9,306,59,60,0,1


In [ ]:
final_dataset.to_excel("hasil_training.xlsx")